In [1]:
import json
import time
import numpy as np
import pandas as pd


In [71]:
fname = "./goodreads_interactions.csv"
start = time.time()
interactions = pd.read_csv(fname)

interactions = interactions.rename(columns={"book_id":"book_id_csv"})
print(time.time() - start)

interactions_book_ids = set(interactions["book_id_csv"].unique())

print(min(interactions_book_ids))
print(max(interactions_book_ids))
print(interactions.shape)
interactions = interactions[(interactions.is_read > 0) & (interactions.is_reviewed > 0)]
print(interactions.shape)


79.27866196632385
0
2360649
(228648342, 5)
(15935237, 5)


In [49]:
fname = './goodreads_books.json'
from io import StringIO
from csv import writer 

output = StringIO()
csv_writer = writer(output)
i = 0
bad = []
start = time.time()
f = open(fname)
for line in f:
    try:
        s = json.loads(line)
    except json.JSONDecodeError:
        bad.append(line)
        continue
#     if(i % 100000 == 0):
#         print(i)
    book_id = s["book_id"]
    title = s['title']
    authors = s['authors']
    average_rating = s['average_rating']
    image_url = s['image_url']
    num_pages = s['num_pages']
    publisher = s['publisher']
    publication_year = s['publication_year']
    ratings_count = s['ratings_count']
    csv_writer.writerow([book_id, title, authors, average_rating, image_url, num_pages, publisher, publication_year, ratings_count])
    i = i + 1

output.seek(0) # we need to get back to the start of the BytesIO
books = pd.read_csv(output, header=None)
books.columns = ['book_id', 'title', 'authors', 'average_rating', 'image_url', 'num_pages', 'publisher', 'publication_year', 'ratings_count']
f.close()
books.to_csv("books.csv")
print(time.time() - start)


120.06343722343445


In [51]:
fname = "./book_id_map.csv"
start = time.time()
book_id_map = pd.read_csv(fname)
print(time.time() - start)

0.3424811363220215


In [60]:
books_book_ids = set(books["book_id"].unique())
print(min(books_book_ids))
print(max(books_book_ids))
print(len(books_book_ids))
books.head()

1
36530431
2360655


,book_id,title,authors,average_rating,image_url,num_pages,publisher,publication_year,ratings_count
0,5333265,W.C. Fields: A Life on Film,"[{'author_id': '604031', 'role': ''}]",4.00,https://images.gr-assets.com/books/1310220028m...,256.0,St. Martin's Press,1984.0,3.0
1,1333909,Good Harbor,"[{'author_id': '626222', 'role': ''}]",3.23,https://s.gr-assets.com/assets/nophoto/book/11...,NaN,Simon & Schuster Audio,2001.0,10.0
2,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","[{'author_id': '10333', 'role': ''}]",4.03,https://images.gr-assets.com/books/1304100136m...,600.0,"Nelson Doubleday, Inc.",1987.0,140.0
3,6066819,Best Friends Forever,"[{'author_id': '9212', 'role': ''}]",3.49,https://s.gr-assets.com/assets/nophoto/book/11...,368.0,Atria Books,2009.0,51184.0
4,287140,Runic Astrology: Starcraft and Timekeeping in ...,"[{'author_id': '149918', 'role': ''}]",3.40,https://images.gr-assets.com/books/1413219371m...,NaN,NaN,NaN,15.0


In [73]:
interactions_book_ids = set(interactions["book_id_csv"].unique())
print(min(interactions_book_ids))
print(max(interactions_book_ids))
print(len(interactions_book_ids))
interactions.head()

0
2360649
2318599


,user_id,book_id_csv,is_read,rating,is_reviewed
1,0,947,1,5,1
9,0,939,1,5,1
10,0,938,1,5,1
14,0,934,1,5,1
18,0,930,1,2,1


In [63]:
book_id_map_id_csv = set(book_id_map["book_id_csv"].unique())
book_id_map_id = set(book_id_map["book_id"].unique())
print(min(book_id_map_id_csv))
print(max(book_id_map_id_csv))
print(len(book_id_map_id_csv))
print(min(book_id_map_id))
print(max(book_id_map_id))
print(len(book_id_map_id))
book_id_map.head()

0
2360649
2360650
1
36530431
2360650


,book_id_csv,book_id
0,0,34684622
1,1,34536488
2,2,34017076
3,3,71730
4,4,30422361


In [74]:
interactions1 = pd.merge(interactions, book_id_map, on="book_id_csv")

In [123]:
interactions1.to_csv("ratings.csv")

In [77]:
book_id_map[book_id_map['book_id_csv'] == 947]

,book_id_csv,book_id
947,947,21


In [80]:
print(len(interactions))
print(len(interactions1))
interactions1

15935237
15935237


,user_id,book_id_csv,is_read,rating,is_reviewed,book_id
0,0,947,1,5,1,21
1,311,947,1,2,1,21
2,373,947,1,2,1,21
3,887,947,1,5,1,21
4,1192,947,1,3,1,21
...,...,...,...,...,...,...
15935232,876136,2360592,1,5,1,36110583
15935233,876136,2360591,1,5,1,36110839
15935234,876136,2360590,1,5,1,36110997
15935235,876136,2360589,1,5,1,36217419


In [110]:
# len(books['authors']) > 0
books['authors'].str.len()
# books['authors'] = books['authors'].apply(lambda x: x.strip('][').split('},'))
books['authors'] = books['authors'].apply(lambda x: [str(y) + '}' for y in x])
# books['authors'] = 
books.dtypes
str1 = books.iat[0, books.columns.get_loc('authors')]
str1 = "[{'author_id': '604031', 'role': ''}, {'author_id': '604032', 'role': ''}]"

print(str1)
res = str1.strip('][').split('},')
[str(x) + '}' for x in res]

[{'author_id': '604031', 'role': ''}, {'author_id': '604032', 'role': ''}]


["{'author_id': '604031', 'role': ''}",
 " {'author_id': '604032', 'role': ''}}"]

In [108]:
print (type(books.iat[0, books.columns.get_loc('authors')]))

<class 'list'>


In [122]:
books.iloc[2360650]["authors"][0]

"{'author_id': '4015', 'role': ''}"